In [1]:
import sys
import pmt
import zmq
import time

import numpy as np

from gnuradio import OFDM_OOT

In [2]:
# ofdm cfg
nfft = 256
frame_sym_cnt = 16

f_norm_min = 0.1
f_norm_max = 0.26

pilot_idx_group_0 = np.arange(f_norm_min*nfft, f_norm_max*nfft, 4, dtype=int)
pilot_idx_group_1 = nfft + np.arange(-f_norm_min*nfft, -f_norm_max*nfft, -4, dtype=int)
pilot_idx = np.hstack([pilot_idx_group_0 , pilot_idx_group_1])

pilot_vals_group_0 = 1.4 * np.exp(-2j*np.pi*np.arange(pilot_idx_group_0.size)/2)
pilot_vals_group_1 = 1.4 * np.exp(-2j*np.pi*np.arange(pilot_idx_group_1.size)/2)
pilot_vals = np.hstack([pilot_vals_group_0, pilot_vals_group_1])

data_idx = np.setdiff1d(np.hstack([np.arange(f_norm_min*nfft, f_norm_max*nfft, dtype=int) , np.arange(-f_norm_min*nfft, -f_norm_max*nfft, -1, dtype=int)]), pilot_idx)[:-1]

In [3]:
zmq_context = zmq.Context()

tx_bytes_socket_addr = "tcp://127.0.0.1:55554"
tx_frame_socket_addr = "tcp://127.0.0.1:55555"
rx_bytes_socket_addr = "tcp://127.0.0.1:55556"
rx_frame_socket_addr = "tcp://127.0.0.1:55557"

tx_bytes_socket = zmq_context.socket(zmq.PUB)
tx_bytes_socket.bind(tx_bytes_socket_addr)

tx_frame_socket = zmq_context.socket(zmq.PUB)
tx_frame_socket.bind(tx_frame_socket_addr)

rx_bytes_socket = zmq_context.socket(zmq.SUB)
rx_bytes_socket.connect(rx_bytes_socket_addr)
rx_bytes_socket.setsockopt(zmq.SUBSCRIBE, b'')

rx_frame_socket = zmq_context.socket(zmq.SUB)
rx_frame_socket.connect(rx_frame_socket_addr)
rx_frame_socket.setsockopt(zmq.SUBSCRIBE, b'')

In [4]:
def send_frame(symbols=None):
    if symbols is None:
        symbols = np.zeros((frame_sym_cnt, nfft), dtype=np.complex64)
        for i in range(frame_sym_cnt):
            symbols[i] = OFDM_OOT.random_ofdm_sym(nfft, data_idx, pilot_idx, pilot_vals, data_modulation='qpsk')
    
    else:
        assert symbols.dtype==np.complex64
    
    tx_frame_socket.send(symbols.flatten())

    
def recv_frame(timeout=None):
    if rx_socket.poll(timeout) != 0:
        byte_data = rx_frame_socket.recv()
        complex_data = np.frombuffer(rx_frame, dtype=np.complex64, count=-1).reshape((-1, nfft))
    else:
        return None
    
    assert complex_data.shape==(frame_sym_cnt, nfft)
    
    return complex_data


def send_bytes(data):
    tx_bytes_socket.send(data)

    
def recv_bytes(timeout=None):
    if rx_socket.poll(timeout) != 0:
        byte_data = rx_bytes_socket.recv()
    else:
        return None
    
    return byte_data

In [5]:
bits_per_iq = 2
bits_per_frame = bits_per_iq*data_idx.size*frame_sym_cnt
assert bits_per_frame%8==0
bytes_per_frame = bits_per_frame//8

frame_data = np.random.randint(0, 2**8, bytes_per_frame, dtype=np.uint8)

In [20]:
tx_bytes_socket.send(frame_data)

In [21]:
rx_data = np.frombuffer(rx_bytes_socket.recv(), dtype=np.uint8, count=-1).reshape((-1, bytes_per_frame))
rx_data.shape

(1, 280)

In [23]:
frame_data-rx_data

array([[ 64,   0, 128, 192,   0, 192,  64,  64,  64,  64,   0,   0,   0,
          0,   0,   0,   0, 244,   4, 252,  12, 244,   8, 252,   4, 248,
          8, 252,   0,   0,   0,   0,   0,   0,   0, 192,   0, 128, 192,
        192,   0, 192,  64,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        244,  12, 244,  12,   0,  12, 244,   4, 252,   8, 252,   0,   0,
          0,   0,   0,   0,   0,   0, 128,  64, 128,  64,   0,   0, 192,
          0, 128, 192,   0,   0,   0,   0,   0,   0,   8,   0,   4,   4,
          8, 252,   8, 252,   0,   0,   8,   0,   0,   0,   0,   0,   0,
          0, 192,   0, 128,  64, 128,   0,   0, 192,   0, 192, 192,   0,
          0,   0,   0,   0,   0, 252,   4, 252, 252,   8, 248,   8, 252,
          4, 252,   8,   0,   0,   0,   0,   0,   0,   0,  64,   0, 128,
         64,   0, 192, 128,  64, 128, 192,  64,   0,   0,   0,   0,   0,
          0, 252,   0,   4, 248,   4, 252, 252,   4,   0,   0,   4,   0,
          0,   0,   0,   0,   0,   0, 128,  64,  64

In [22]:
total = 0
for diff in frame_data - rx_data[0]:
    total += bin(diff).count('1')

print(total)
print(total/(bits_per_frame))

328
0.14642857142857144


In [14]:
bin(71)

'0b1000111'

In [12]:
rx_data[0]

array([0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0,
       0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3,
       0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 2, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [13]:
frame_data

array([ 71, 229, 132, 118,  89, 192,  88,  98, 228,   5, 168, 139, 177,
        72,   7,  70, 135,  21,  33,  82,  68, 135,  94,  73,  30,  59,
       157,  84,  86,  27, 226,  96, 247,  99, 165, 163,  61, 143,  24,
        57,  66, 128,  43,  82, 150, 186, 122, 135,  40,  25, 182,  96,
       138, 219, 229, 225,  84, 226,  59, 142, 204,  71,  90,  42, 164,
       235,  87,  64, 160, 199,  29,  10, 235, 254,  30, 229,  30, 190,
       120, 153, 160, 183,  54, 176, 155,  41, 246,  12, 106,  47, 114,
        69, 229,  71, 114,  83,  19, 157, 242,  84,  91, 181,  51,  16,
       164, 170, 224,  53, 181,  73, 181, 132, 228, 162, 150,  54, 174,
       106,  16,   8, 227,  66, 196, 164, 115, 103,  36,  16, 180, 115,
        44, 128,  88,  31,  41, 174,   4, 232, 144, 123, 176, 235, 177,
       127,  43,  71, 182, 171, 159,  55, 143, 171,  81,  76,  14, 179,
        99,  91,  93,  23, 228, 206, 175,  43, 138, 138, 145, 186, 208,
       150, 233, 255, 202, 146,  46, 140, 245, 201, 241,  90,  9

In [19]:
rx_data

array([[0, 0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 3,
        0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 2, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0,
        0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 1]], dtype=uint8)

In [13]:
send_bytes(frame_data)

In [18]:
for i in range(20):
    send_frame()
    time.sleep(0.01)

In [41]:
rx_frame_sym = receive_frame()

In [32]:
data = np.frombuffer(rx_frame, dtype=np.complex64, count=-1)

In [33]:
data.shape[0]/nfft

16.0

In [1]:
import numpy as np
np.set_printoptions(precision=3)
from matplotlib import pyplot as plt

from gnuradio import OFDM_OOT
from gnuradio import analog
from gnuradio import blocks
from gnuradio import digital
from gnuradio import filter
from gnuradio import gr
from gnuradio import iio

from PyQt5 import Qt
from gnuradio import qtgui

In [2]:
class pluto_tx_flowgraph():
    def __init__(self, fc=int(2.013e9), fs=int(1e6)):
        self.iio_pluto_sink = iio.fmcomms2_sink_fc32('ip:192.168.2.1', 
                                                     [True, True], 
                                                     int(fs/10), # buffer size
                                                     True)
        self.iio_pluto_sink.set_frequency(fc)
        self.iio_pluto_sink.set_samplerate(fs)
        self.iio_pluto_sink.set_bandwidth(2*fs)
        self.iio_pluto_sink.set_attenuation(0, 10)
        self.iio_pluto_sink.set_filter_params('Auto', '', 0, 0)
        self.iio_pluto_sink.set_len_tag_key('')
        
    def start(self, iq):
        tb = gr.top_block()
        
        src = blocks.vector_source_c(tuple(iq))
        tb.connect((src, 0), (self.iio_pluto_sink, 0))
        tb.start()
        

In [3]:
tx = pluto_tx_flowgraph(fc=int(2e9), fs=int(1e6))

In [4]:
nfft = 1024
ng = 128
iq = np.random.choice([1., -1.], nfft) + 1j*np.random.choice([1., -1.], nfft)
iq[0] = 0
iq_t = np.fft.ifft(iq)
iq_t = np.hstack((iq_t[-ng:], iq_t))

In [54]:

while(True):
    tx.start(iq_t*100)

KeyboardInterrupt: 

In [53]:

tx.start(iq_t*1000)